In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Hp\\Desktop\\Image_Captioning_End_to_End_Deployment'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from src.Model.constants import *
from src.Model.utils.common import read_yaml, create_directories
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [7]:
import time

In [8]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [1]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from tqdm import tqdm
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
from nltk.translate.bleu_score import corpus_bleu
import numpy as np

ModuleNotFoundError: No module named 'nltk'

In [20]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    

    def cleaning(self, mapping):

        for key, captions in mapping.items():
            for i in range(len(captions)):
                caption = captions[i]
                caption = caption.lower()
                caption = caption.replace('[^A-Za-z]', '')
                caption = caption.replace('\s+', ' ')
                caption = 'startseq ' + " ".join([word for word in caption.split() if len(word)>1]) + ' endseq'
                captions[i] = caption
    

    def train_valid_generator(self):
        with open(os.path.join("artifacts/data_ingestion/", 'features.pkl'), 'rb') as f:
            self.features = pickle.load(f)
        
        with open(os.path.join("", 'captions.txt'), 'r') as f:
            next(f)
            captions_doc = f.read()
        
        self.mapping = {}

        for line in tqdm(captions_doc.split('\n')):
            # split the line by comma(,)
            tokens = line.split(',')
            if len(line) < 2:
                continue
            image_id, caption = tokens[0], tokens[1:]
            # remove extension from image ID
            image_id = image_id.split('.')[0]

            caption = " ".join(caption)
            # create list if needed
            if image_id not in self.mapping:
                self.mapping[image_id] = []
            # store the caption
            self.mapping[image_id].append(caption)
        
        self.cleaning(self.mapping)

        all_captions = []
        for key in self.mapping:
            for caption in self.mapping[key]:
                all_captions.append(caption)
        
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(all_captions)
        self.vocab_size = len(self.tokenizer.word_index) + 1

        pickle.dump(self.tokenizer, open(os.path.join("artifacts/training/", 'tokenize.pkl'), 'wb'))
    

    def data_generator(self, data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
        X1, X2, y = list(), list(), list()
        n = 0
        while 1:
            for key in data_keys:
                n += 1
                captions = mapping[key]
                for caption in captions:
                    seq = tokenizer.texts_to_sequences([caption])[0]
                    for i in range(1, len(seq)):
                        in_seq, out_seq = seq[:i], seq[i]
                        in_seq = pad_sequences([in_seq], maxlen=max_length, padding='post')[0]
                        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                        X1.append(features[key][0])
                        X2.append(in_seq)
                        y.append(out_seq)
                if n == batch_size:
                    X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                    yield {"image": X1, "text": X2}, y
                    X1, X2, y = list(), list(), list()
                    n = 0
        

        

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):

        image_ids = list(self.mapping.keys())
        split = int(len(image_ids) * 0.90)
        train = image_ids[:split]
        self.steps_per_epoch = len(train) // self.config.params_batch_size

        self.model.fit(
            self.data_generator(
                data_keys=train,
                mapping=self.mapping,
                features=self.features,
                tokenizer=self.tokenizer,
                max_length=35,
                vocab_size=self.vocab_size,
                batch_size=self.config.params_batch_size
            ),
            epochs=1,
            steps_per_epoch=self.steps_per_epoch,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [21]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-06-08 11:02:33,762 - INFO - common - yaml file: config\config.yaml loaded successfully]
[2025-06-08 11:02:33,765 - INFO - common - yaml file: params.yaml loaded successfully]
[2025-06-08 11:02:33,766 - INFO - common - created directory at: artifacts]
[2025-06-08 11:02:33,767 - INFO - common - created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-06-08 11:02:33,769 - INFO - common - created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-06-08 11:02:33,770 - INFO - common - created directory at: artifacts\training]


c:\Users\Hp\Desktop\Image_Captioning_End_to_End_Deployment\venv\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
100%|██████████| 40456/40456 [00:00<00:00, 1160912.41it/s]


113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 6.1914

c:\Users\Hp\Desktop\Image_Captioning_End_to_End_Deployment\venv\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:302: UserWarning: Can save best model only with val_loss available.
  if self._should_save_model(epoch, batch, logs, filepath):


113/113 ━━━━━━━━━━━━━━━━━━━━ 342s 3s/step - loss: 6.1859
